In [1]:
import tensorflow as tf
from tensorflow import keras

In [9]:
# mnist， 手写数字识别 （28，28，1）
(x,y),(x_test,y_test)=keras.datasets.mnist.load_data()
print(x.shape,x_test.shape, x.min(),x.max(),x.mean())
# check y label
print(y[1:10])
y_onehot=tf.one_hot(y,depth=10)
print(y_onehot[1])

(60000, 28, 28) (10000, 28, 28) 0 255 33.318421449829934
[0 4 1 9 2 1 3 1 4]
tf.Tensor([1. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float32)


In [28]:
#cifar10/100 简单的图片识别 （32，32，3）
(x,y),(x_test,y_test)=keras.datasets.cifar10.load_data()
print(x.shape,y.shape,x_test.shape)
print(x.min(),x.max())
print(y[:3])

# 将numpy 转化为tensor
db=tf.data.Dataset.from_tensor_slices(x_test)
itdb=iter(db) # 迭代器
print(next(itdb).shape)

#多个numpy转化为tensor
db=tf.data.Dataset.from_tensor_slices((x_test,y_test))
itdb=iter(db)
print(next(itdb)[0].shape)
print(next(itdb)[1].shape)

# shuffle 数据
db=db.shuffle(1000)

# 数据位数转化，numpy默认是64, tensorflow 是32
def preprocess(x,y):
    x=tf.cast(x,dtype=tf.float32)/255
    y=tf.cast(y,dtype=tf.int32)
    y=tf.one_hot(y,depth=10)
    return x,y

db2=db.map(preprocess)
itdb2=iter(db2)
print(next(itdb2)[0].shape)
print(next(itdb2)[1].shape)
print(next(itdb2)[1][:3])

# batch 
db3=db2.batch(32)
itdb3=iter(db3)
print(next(itdb3)[0].shape)
print(next(itdb3)[1].shape)

num=0
for num_x_test,num_y_test in itdb3:
    num=num+1
print(num)
    
# repeat
db4=db3.repeat() # 不会停止
itdb4=iter(db4)
num=0
while True:
    it_x_test,it_y_test=next(itdb4)
    num=num+1
print(num)

# 会出现迭代错误, 需要catch error, 然后重新给一个迭代器
num=0
while True:
    it_x_test,it_y_test=next(itdb3)
    num=num+1
print(num)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3)
0 255
[[6]
 [9]
 [9]]
(32, 32, 3)
(32, 32, 3)
(1,)
(32, 32, 3)
(1, 10)
tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]], shape=(1, 10), dtype=float32)
(32, 32, 32, 3)
(32, 1, 10)


KeyboardInterrupt: 

In [ ]:
# 完整流程
def prepare_mnist_features_and_labels(x,y):
    x=tf.cast(x,dtype=tf.float32)/255.0
    y=tf.cast(y,dtype=tf.int64)
    return x,y

def mnist_dataset():
    (x,y),(x_val,y_val)=keras.datasets.fashion_mnist.load_data()
    y=tf.one_hot(y,depth=10)
    y_val=tf.one_hot(y_val,depth=10)
    
    ds=tf.data.Dataset.from_tensor_slices((x,y))
    ds=ds.map(prepare_mnist_features_and_labels)
    ds=ds.shuffle(1000).batch(100)
    
    ds_val=tf.data.Dataset.from_tensor_slices((x_val,y_val))
    ds_val=ds_val.map(prepare_mnist_features_and_labels)
    ds_val=ds_val.shuffle(1000).batch(100)
    
    returen ds, ds_val
